In [ ]:
""" Fizz Buzz Example, Comparing tf.keras and keras """

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
""" Create training and validation samples """

n_training = 1000
n_bin_encode = 10 # max is 2^p
n_valid = 100
train_list = []
train_label = []
valid_list = []
valid_label = []

def bin_encode(i):
    # encode the digit to binary-format
    return [i >> d & 1 for d in range(n_bin_encode)]

def label_fizz_buss(x):
    # fizz (3) = 1
    # buzz (5) = 2
    # fizz_buzz (15) = 3
    # neither = 0
    if x % 15 == 0:
        return 3
    elif x % 5 == 0:
        return 2
    elif x % 3 == 0:
        return 1
    else:
        return 0

#for i in np.arange(n_training+1)[1:]:
for i in np.arange(101, 901):
    train_list.append(bin_encode(i))
    train_label.append(label_fizz_buss(i))


#for i in np.arange(n_training+n_valid+1)[n_training+1:]:
for i in np.arange(901,1000):
    valid_list.append(bin_encode(i))
    valid_label.append(label_fizz_buss(i))


# Convert dataset to array
train_list = np.array(train_list, dtype=np.float32)
valid_list = np.array(valid_list, dtype=np.float32)
# Convert labels to one-hot encoding
train_label = tf.keras.utils.to_categorical(train_label)
valid_label = tf.keras.utils.to_categorical(valid_label)

print(train_list[0])

[ 1.  0.  1.  0.  0.  1.  1.  0.  0.  0.]


In [4]:
""" Build the model with tf-keras-1 """

x_in = tf.keras.layers.Input(shape=(n_bin_encode,))

x = tf.keras.layers.Dense(1000, activation='relu')(x_in)
y_out = tf.keras.layers.Dense(4, activation='softmax')(x)

model1 = tf.keras.models.Model(inputs=[x_in], outputs=[y_out])  

my_optim = tf.keras.optimizers.Adam(lr = 0.0001) # <----------- ISSUE!!! if use this ... you will fail...
model1.compile(loss = 'categorical_crossentropy', 
               optimizer = my_optim, 
               metrics=['acc'])
model1.summary()

model1.fit(x=train_list, y=train_label,
          epochs=100, batch_size=20,
          validation_data=(valid_list, valid_label))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              11000     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 4004      
Total params: 15,004
Trainable params: 15,004
Non-trainable params: 0
_________________________________________________________________
Train on 800 samples, validate on 99 samples
Epoch 1/100
800/800 [==============================] - 4s - loss: 1.2962 - acc: 0.4688 - val_loss: 1.2095 - val_acc: 0.5354
Epoch 2/100
800/800 [==============================] - 0s - loss: 1.2025 - acc: 0.5337 - val_loss: 1.1541 - val_acc: 0.5354
Epoch 3/100
800/800 [==============================] - 0s - loss: 1.1654 - acc: 0.5338 - val_loss: 1.1389 - v

In [5]:
""" 
Build the model with tf-keras-2 
Now, we know that using tf.keras.optimizers will fail (the get_update don't work!)
we change the optimizer to tf.train.optimizer
"""

x_in = tf.keras.layers.Input(shape=(n_bin_encode,))

x = tf.keras.layers.Dense(1000, activation='relu')(x_in)
y_out = tf.keras.layers.Dense(4, activation='softmax')(x)

model2 = tf.keras.models.Model(inputs=[x_in], outputs=[y_out])  

""" Also, we make our loss more flexible"""
def my_loss(y_true, y_pred):
    return tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

model2.compile(loss = my_loss, 
               optimizer = tf.train.AdamOptimizer(learning_rate=0.001), 
               metrics=['acc'])
model2.summary()

model2.fit(x=train_list, y=train_label,
          epochs=200, batch_size=20,
          validation_data=(valid_list, valid_label))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              11000     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 4004      
Total params: 15,004
Trainable params: 15,004
Non-trainable params: 0
_________________________________________________________________
Train on 800 samples, validate on 99 samples
Epoch 1/200
800/800 [==============================] - 0s - loss: 1.2553 - acc: 0.5225 - val_loss: 1.2091 - val_acc: 0.5354
Epoch 2/200
800/800 [==============================] - 0s - loss: 1.2105 - acc: 0.5338 - val_loss: 1.2087 - val_acc: 0.5354
Epoch 3/200
800/800 [==============================] - 0s - loss: 1.2100 - acc: 0.5338 - val_loss: 1.2085 - v

In [6]:
""" Now we use keras """
import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam

k_x_in = Input(shape=(n_bin_encode,))

mx = Dense(1000, activation='relu')(k_x_in)
m_out = Dense(4, activation='softmax')(mx)

model3 = Model(inputs=[k_x_in], outputs=[m_out])

k_optim = Adam(lr=0.001)
model3.compile(loss = 'categorical_crossentropy', optimizer = k_optim, metrics=['acc'])

model3.summary()

model3.fit(x=train_list, y=train_label, 
          epochs=100, batch_size=20,
          validation_data=(valid_list, valid_label))

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              11000     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 4004      
Total params: 15,004
Trainable params: 15,004
Non-trainable params: 0
_________________________________________________________________
Train on 800 samples, validate on 99 samples
Epoch 1/100
800/800 [==============================] - 0s - loss: 1.1906 - acc: 0.5225 - val_loss: 1.1413 - val_acc: 0.5354
Epoch 2/100
800/800 [==============================] - 0s - loss: 1.1568 - acc: 0.5337 - val_loss: 1.1307 - val_acc: 0.5354
Epoch 3/100
800/800 [==============================] - 0s - loss: 1.1444 - acc: 0.5337 - val_loss: 1.1287 - v

In [ ]:
test_list = [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007]
test_list = np.array([bin_encode(i) for i in test_list])

In [ ]:
predict = model.predict(test_list)
predict.argmax(axis=1)

In [ ]:
k_predict = k_model.predict(test_list)
k_predict.argmax(axis = 1)